<a href="https://colab.research.google.com/github/nghianh993/BDS/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive

drive.mount('/content/drive')

# Get data
url = "https://rongbachkim.com"
now = datetime.now()
data = []
minusDay = 60
next_date = now + timedelta(days=minusDay)
totalCount = 4
# File name
current_file_name = '/content/drive/MyDrive/Model/AI_239days_20240306.h5'
file_in_drive = '/content/drive/MyDrive/Model/AI_' + str((minusDay * totalCount - 1)) + 'days_' + now.strftime("%Y%m%d") + '.h5'
for number in range(1, totalCount):
    if number == 1:
        next_date = next_date - timedelta(days=minusDay + 1)
    else:
        next_date = next_date - timedelta(days=minusDay)
    # Send an HTTP request to the API and receive JSON data
    response = requests.get(url + '/ketqua.php?getkq&ngay=' + next_date.strftime("%Y-%m-%d") + '&days='+ str(minusDay) +'&wday=0')
    data = data + response.json()

print(data)

X_train = []
y_train = []

for i in range(len(data)-1):
    current_day = list(map(float, data[i][1]))
    next_day = list(map(float, data[i+1][1]))
    X_train.append(current_day)
    y_train.append(next_day)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

## Create and train the model
#model = Sequential()
#model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
#model.add(Dropout(0.2))
#model.add(LSTM(units=128, return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(units=128))
#model.add(Dense(units=1))

model = tf.keras.models.load_model(current_file_name)

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=300, batch_size=1)

# Load the best saved model
model.save(file_in_drive)

last_day = list(map(float, data[0][1]))
print("Dữ liệu data: ", data[0][1])

X_test = np.array([last_day])  # Change here to create a single-element array
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape the test data

predictions = model.predict(X_test)
predictions = np.array(predictions).reshape(predictions.shape[0], predictions.shape[1])  # Reshape predictions to match input format
predictions = predictions.tolist()

print("Kết quả dự đoán:", predictions)